In [1]:
!pip install nnfs

In [2]:
import numpy as np 
import nnfs 
from nnfs.datasets import spiral_data

nnfs.init()

In [3]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons, weight_regularizer_l1=0, weight_regularizer_l2=0,bias_regularizer_l1 = 0,bias_regularizer_l2 = 0):
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2
        
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(self.inputs, self.weights ) + self.biases
        return self.output

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis = 0, keepdims = True)

        if self.weight_regularizer_l1 > 0: 
            dL1 = np.ones_like(self.weights) 
            dL1[self.weights < 0] = -1 
            self.dweights += self.weight_regularizer_l1 * dL1 
  
        if self.weight_regularizer_l2 > 0: 
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights 

        if self.bias_regularizer_l1 > 0: 
            dL1 = np.ones_like(self.biases) 
            dL1[self.biases < 0] = -1 
            self.dbiases += self.bias_regularizer_l1 * dL1 
        if self.bias_regularizer_l2 > 0: 
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases 
    
        self.dinputs = np.dot(dvalues, self.weights.T)

In [4]:
class Layer_Dropout:
    def __init__(self, rate):
        self.rate = 1 - rate

    def forward(self, inputs):
        self.inputs = inputs
        self.binary_mask = np.random.binomial(1,self.rate, size = inputs.shape) / self.rate
        self.output = inputs * self.binary_mask

    def backward(self, dvalues):
        self.dinputs = dvalues * self.binary_mask

In [5]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

In [6]:
class Activation_Softmax:
    def forward(self, inputs):
        exponantial = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        normalize = exponantial / np.sum(exponantial, axis=1, keepdims=True)
        self.output = normalize
        return self.output

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)

        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1,1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix,single_dvalues)


In [7]:
class Optimizer_SGD:
    def __init__(self, learning_rate =1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
                
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates

        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1

In [8]:
class Optimizer_Adagrad:
    def __init__(self, learning_rate =1., decay=0., epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.biases_cache = np.zeros_like(layer.biases) 

        layer.weight_cache += layer.dweights**2
        layer.biases_cache += layer.dbiases**2 

        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.biases_cache) + self.epsilon) 
    
    def post_update_params(self):
        self.iterations += 1

In [9]:
class Optimizer_RMSprop:
    def __init__(self, learning_rate =1., decay=0., epsilon=1e-7, rho = 0.9):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.rho = rho
    
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.biases_cache = np.zeros_like(layer.biases) 

        layer.weight_cache = self.rho * layer.weight_cache + (1-self.rho) * layer.dweights**2
        layer.biases_cache = self.rho * layer.biases_cache + (1-self.rho) * layer.dbiases**2 

        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.biases_cache) + self.epsilon) 
    
    def post_update_params(self):
        self.iterations += 1

In [10]:
class Optimizer_Adam:
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7,beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):

        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

        
        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))

        
        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2
        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2

        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

        
        layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1


In [11]:
class Loss:

    def regularization_loss(self, layer):
        regularization_loss = 0
        if layer.weight_regularizer_l1 > 0:
            regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))
        
        if layer.weight_regularizer_l2 > 0:
            regularization_loss += layer.weight_regularizer_l2 * np.sum((layer.weights * layer.weights))


        if layer.bias_regularizer_l1 > 0:
            regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))
        
        if layer.bias_regularizer_l2 > 0:
            regularization_loss += layer.bias_regularizer_l2 * np.sum((layer.biases * layer.biases))

        return regularization_loss

    
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss


class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]

        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped*y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
    
        if len(y_true.shape) == 1:
            y_true = np.eye(dvalues.shape[1])[y_true]
    
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


In [12]:
class Activation_Softmax_Loss_CategoricalCrossentropy(): 
 
    def __init__(self): 
        self.activation = Activation_Softmax() 
        self.loss = Loss_CategoricalCrossentropy() 
  
    def forward(self, inputs, y_true): 
        self.activation.forward(inputs) 
        self.output = self.activation.output 
        return self.loss.calculate(self.output, y_true) 
 
    def backward(self, dvalues, y_true): 
        samples = len(dvalues) 
 
        if len(y_true.shape) == 2: 
            y_true = np.argmax(y_true, axis=1) 
 
        self.dinputs = dvalues.copy() 
        self.dinputs[range(samples), y_true] -= 1 
        self.dinputs = self.dinputs / samples

In [13]:
X, y = spiral_data(samples=100, classes=3)

dense1 = Layer_Dense(2, 64)

activation1 = Activation_ReLU()

dropout1 = Layer_Dropout(0.1)

dense2 = Layer_Dense(64, 3, weight_regularizer_l2=1e-3, bias_regularizer_l2=1e-3)

loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()


optimizer_name = "adam"  # "sgd", "adagrad", "rmsprop", "adam"

if optimizer_name == "sgd":
    optimizer = Optimizer_SGD(learning_rate=0.05, decay=5e-7, momentum=0.9)

elif optimizer_name == "adagrad":
    optimizer = Optimizer_Adagrad(learning_rate=1.0, decay=5e-7)

elif optimizer_name == "rmsprop":
    optimizer = Optimizer_RMSprop(learning_rate=0.001, decay=5e-7, rho=0.9)

elif optimizer_name == "adam":
    optimizer = Optimizer_Adam(learning_rate=0.05, decay=5e-7)


for epoch in range(10001):

    dense1.forward(X)

    activation1.forward(dense1.output)

    dropout1.forward(activation1.output)

    dense2.forward(dropout1.output)

    data_loss = loss_activation.forward(dense2.output, y)

    regularization_loss = loss_activation.loss.regularization_loss(dense1) + loss_activation.loss.regularization_loss(dense2) 

    loss = data_loss + regularization_loss

    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')

    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    dropout1.backward(dense2.dinputs) 
    activation1.backward(dropout1.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.347, loss: 1.099, lr: 0.05
epoch: 100, acc: 0.600, loss: 0.947, lr: 0.04999752512250644
epoch: 200, acc: 0.620, loss: 0.950, lr: 0.04999502549496326
epoch: 300, acc: 0.650, loss: 0.868, lr: 0.049992526117345455
epoch: 400, acc: 0.610, loss: 0.883, lr: 0.04999002698961558
epoch: 500, acc: 0.663, loss: 0.847, lr: 0.049987528111736124
epoch: 600, acc: 0.650, loss: 0.814, lr: 0.049985029483669646
epoch: 700, acc: 0.660, loss: 0.832, lr: 0.049982531105378675
epoch: 800, acc: 0.640, loss: 0.830, lr: 0.04998003297682575
epoch: 900, acc: 0.710, loss: 0.771, lr: 0.049977535097973466
epoch: 1000, acc: 0.690, loss: 0.806, lr: 0.049975037468784345
epoch: 1100, acc: 0.690, loss: 0.775, lr: 0.049972540089220974
epoch: 1200, acc: 0.690, loss: 0.779, lr: 0.04997004295924593
epoch: 1300, acc: 0.690, loss: 0.792, lr: 0.04996754607882181
epoch: 1400, acc: 0.647, loss: 0.779, lr: 0.049965049447911185
epoch: 1500, acc: 0.680, loss: 0.799, lr: 0.04996255306647668
epoch: 1600, acc: 0.667, lo

In [14]:
X, y = spiral_data(samples=100, classes=3)
dense1.forward(X)
activation1.forward(dense1.output) 
dense2.forward(activation1.output) 
loss = loss_activation.forward(dense2.output, y)
predictions  = np.argmax(loss_activation.output, axis=1) 
if  len(y.shape)  ==  2: 
    y  =  np.argmax(y,  axis=1) 
accuracy  =  np.mean(predictions==y) 
print(f'validation, acc:  {accuracy:.3f}, loss:  {loss:.3f}')

validation, acc:  0.660, loss:  0.777
